## How to break an Enigma machine like Rejewski

This notebook will walk you through the process of recovering the day key settings for an Enigma machine. 

Before you get started, here's what you need to know: 
  - You will be working with the 3 rotor Wehrmacht Enigma machine. Read more about it [here](http://users.telenet.be/d.rijmenants/en/enigmatech.htm). You will only consider rotors I, II, and III for the purposes of this exercise. 
  - The dictionary of cycle lengths has already been generated for you. You will access it later. 
  - There's a little bit of historical anachronism in the scenario described, since Rejewski's method of cracking Enigma was developed long before Bletchley Park existed, and codebreakers at Bletchley Park used more advanced methods than did Rejewski. Just roll with it. 

### The Scenario. 

It's midnight at Bletchley, the middle of your nighttime shift. The past days of German army traffic you decrypted have hinted at some major events to come today. Since the new day just started, you know that the German army Enigma operators have just switched over to a new day key, rendering the previous day's work decrypting their traffic effectively useless. Given the rumors swirling about a potential German attack today, you know you need to crack this new day key as soon as possible. 

Shortly after midnight, freshly encrypted German messages begin piling up on your desk, demanding your attention. You know what to do. Let's get cracking. 

### Materials

You have at your hands the following materials: 
- A huge set of German messages encrypted with the unknown day key (which includes the rotor key, plugboard settings, and rotor order). They are stored in the file 'message_key_encrypts.pickle'. For your conveniences, only the first 6 letters from each of these encrypts (the twice-enciphered message key) have been kept. 
- A dictionary of all the different chain lengths generated by every possible rotor key and order combination. 
- A small test set of messages encrypted with the day key rotor/plugboard settings and the message key rotor position associated with the first item in the message_key_encrypts list.  
- Also you have a computer. You can certainly do parts of this exercise manually, but please please feel free to write some code to supplement your work. After all, breaking Enigma took Rejewski over a year. You have an hour. 

### A Note on Jupyter Notebooks
This is a Jupyter notebook. It allows the mingling of code and text. This notebook is written using Python 2.

To run the Python code in a cell, use Control + Enter/Return. 

To edit Python code in a cell, simply click into that cell and type. 

### A Brief Enigma Overview
The below code will walk you through how to use the Enigma simulator written for this exercise. This knowledge should help you as you figure out how to crack Enigma.

In [1]:
# Import the Enigma module. 
# REMEMBER: to run a particular cell in a Jupyter notebook, press Ctrl + Enter.
import machine as m

In [2]:
# Create an Enigma machine. 
# Since you don't specify any settings, this Enigma is created with the initial rotor position setting 'AAA'
# and rotor order I, II, III.
e = m.Enigma()

In [3]:
# Set the Enigma machine rotor position so that you are sure it is initialized for encryption. 
e.set_rotor_position('AAA')
# Now encrypt.
e.encipher('this is a test')

'ZPJJSVSPGBW'

In [4]:
# Now reset the rotor position so that it is initialized for decryption. 
e.set_rotor_position('AAA')
# Now decrypt. 
e.decipher('zpjjsvspgbw')

'THISISATEST'

Note that you can use the e.enicpher and e.decipher interchangably, since encryption and decryption are the same. However, these two separate functions are provided for your convenience. 

### Now, let's break Enigma.

In [5]:
# First, import all the modules you will need.
import machine as m
import pickle

In [6]:
# Next, load in the message key encrypts. Recall that these are the rotor positions for a particular message that have been enciphered twice with the day key settings. 
message_key_encrypts = pickle.load(open('message_key_encrypts.pickle', 'rb'))
# Take a look at the first 30 message key encrypts. 
message_key_encrypts[0:10]

[u'ZEUATW',
 u'MVBXPQ',
 u'LSCEJY',
 u'GYICYG',
 u'ORZNBX',
 u'UZBBXQ',
 u'LCHEKP',
 u'DMGOOO',
 u'UQVBDR',
 u'BPZSIX']

Following Rejewski's original attack, you will need to generate the AD, BE, and CF permutations. Recall that, if we think of the twice-encrypted message key as ABCDEF, we know that the first and third (A and D), second and fourth (B and E), and third and fifth (C and F) letters in the encryption are generated by the same plaintext letter. Because we know this intimate relationship between AD, BE, and CF, we can generate permutation maps for these three pairs. 

Take the following 5 message encrypts for example: 'ZEUATW', 'MVBXPQ', 'LSCEJY', 'GYICYG', and 'ORZNBX'. Because of the known relationship between the first and fourth letters (A and D) in these encrypts, we find the following permutation mappings: 

- AD: Z -> A, M -> X, L -> E, G -> C, O -> N. 

Similarly, for BE and CF, we find:
 
- BE: E -> T, V -> P, S -> J, Y -> Y, R -> B (a letter mapping to itself is ok in this scenario because this is a permutation map, not a direct map). 
- CF: U -> W, B -> Q, C -> Y, I -> G, Z -> X 

If we were to repeat this process until we found a mapping for every letter in the alphabet for each of the AD, BE, and CF permuations, we could figure out the "chain lengths" generated by these permutation maps. If you're fuzzy on the cycle length description, take a look at slides 150-180 found [here](https://www.math.ias.edu/files/wam/enigma_Treatman.pdf).

In [7]:
###Creates a dictionary of the permuations
D14={}
D25={}
D36={}
#python starts counting at 0
for mes in message_key_encrypts:
    D14[mes[0]]=mes[3]
    D25[mes[1]]=mes[4]
    D36[mes[2]]=mes[5]

In [8]:
D14

{u'A': u'H',
 u'B': u'S',
 u'C': u'G',
 u'D': u'O',
 u'E': u'L',
 u'F': u'R',
 u'G': u'C',
 u'H': u'K',
 u'I': u'Q',
 u'J': u'Y',
 u'K': u'V',
 u'L': u'E',
 u'M': u'X',
 u'N': u'W',
 u'O': u'N',
 u'P': u'U',
 u'Q': u'M',
 u'R': u'F',
 u'S': u'T',
 u'T': u'I',
 u'U': u'B',
 u'V': u'D',
 u'W': u'Z',
 u'X': u'P',
 u'Y': u'J',
 u'Z': u'A'}

In [9]:
D36

{u'A': u'J',
 u'B': u'Q',
 u'C': u'Y',
 u'D': u'A',
 u'E': u'D',
 u'F': u'C',
 u'G': u'O',
 u'H': u'P',
 u'I': u'G',
 u'J': u'I',
 u'K': u'S',
 u'L': u'B',
 u'M': u'V',
 u'N': u'L',
 u'O': u'E',
 u'P': u'U',
 u'Q': u'K',
 u'R': u'T',
 u'S': u'N',
 u'T': u'Z',
 u'U': u'W',
 u'V': u'R',
 u'W': u'F',
 u'X': u'M',
 u'Y': u'H',
 u'Z': u'X'}

In [10]:
D25

{u'A': u'Q',
 u'B': u'A',
 u'C': u'K',
 u'D': u'V',
 u'E': u'T',
 u'F': u'G',
 u'G': u'F',
 u'H': u'W',
 u'I': u'S',
 u'J': u'R',
 u'K': u'C',
 u'L': u'Z',
 u'M': u'O',
 u'N': u'N',
 u'O': u'H',
 u'P': u'I',
 u'Q': u'D',
 u'R': u'B',
 u'S': u'J',
 u'T': u'U',
 u'U': u'L',
 u'V': u'P',
 u'W': u'E',
 u'X': u'M',
 u'Y': u'Y',
 u'Z': u'X'}

In [11]:
###Take D14,D25, and D36 and write them in cycle structure. 
def dict_to_cycle(D,n):
    """
    Input: dictionary D, number in the list
    Output: list of lists
    """
    cycle=[]
    Keys=D.keys()
    key=Keys[n]
    cycle.append(key)
    next_key=D[key]
    while next_key not in cycle:
        key=next_key
        cycle.append(key)
        next_key=D[key]
    return cycle    

In [12]:
for n in range(26):
    print dict_to_cycle(D14,n)

[u'A', u'H', u'K', u'V', u'D', u'O', u'N', u'W', u'Z']
[u'C', u'G']
[u'B', u'S', u'T', u'I', u'Q', u'M', u'X', u'P', u'U']
[u'E', u'L']
[u'D', u'O', u'N', u'W', u'Z', u'A', u'H', u'K', u'V']
[u'G', u'C']
[u'F', u'R']
[u'I', u'Q', u'M', u'X', u'P', u'U', u'B', u'S', u'T']
[u'H', u'K', u'V', u'D', u'O', u'N', u'W', u'Z', u'A']
[u'K', u'V', u'D', u'O', u'N', u'W', u'Z', u'A', u'H']
[u'J', u'Y']
[u'M', u'X', u'P', u'U', u'B', u'S', u'T', u'I', u'Q']
[u'L', u'E']
[u'O', u'N', u'W', u'Z', u'A', u'H', u'K', u'V', u'D']
[u'N', u'W', u'Z', u'A', u'H', u'K', u'V', u'D', u'O']
[u'Q', u'M', u'X', u'P', u'U', u'B', u'S', u'T', u'I']
[u'P', u'U', u'B', u'S', u'T', u'I', u'Q', u'M', u'X']
[u'S', u'T', u'I', u'Q', u'M', u'X', u'P', u'U', u'B']
[u'R', u'F']
[u'U', u'B', u'S', u'T', u'I', u'Q', u'M', u'X', u'P']
[u'T', u'I', u'Q', u'M', u'X', u'P', u'U', u'B', u'S']
[u'W', u'Z', u'A', u'H', u'K', u'V', u'D', u'O', u'N']
[u'V', u'D', u'O', u'N', u'W', u'Z', u'A', u'H', u'K']
[u'Y', u'J']
[u'X', u'P', u'U

In [13]:
for n in range(26):
    print dict_to_cycle(D25,n)

[u'A', u'Q', u'D', u'V', u'P', u'I', u'S', u'J', u'R', u'B']
[u'C', u'K']
[u'B', u'A', u'Q', u'D', u'V', u'P', u'I', u'S', u'J', u'R']
[u'E', u'T', u'U', u'L', u'Z', u'X', u'M', u'O', u'H', u'W']
[u'D', u'V', u'P', u'I', u'S', u'J', u'R', u'B', u'A', u'Q']
[u'G', u'F']
[u'F', u'G']
[u'I', u'S', u'J', u'R', u'B', u'A', u'Q', u'D', u'V', u'P']
[u'H', u'W', u'E', u'T', u'U', u'L', u'Z', u'X', u'M', u'O']
[u'K', u'C']
[u'J', u'R', u'B', u'A', u'Q', u'D', u'V', u'P', u'I', u'S']
[u'M', u'O', u'H', u'W', u'E', u'T', u'U', u'L', u'Z', u'X']
[u'L', u'Z', u'X', u'M', u'O', u'H', u'W', u'E', u'T', u'U']
[u'O', u'H', u'W', u'E', u'T', u'U', u'L', u'Z', u'X', u'M']
[u'N']
[u'Q', u'D', u'V', u'P', u'I', u'S', u'J', u'R', u'B', u'A']
[u'P', u'I', u'S', u'J', u'R', u'B', u'A', u'Q', u'D', u'V']
[u'S', u'J', u'R', u'B', u'A', u'Q', u'D', u'V', u'P', u'I']
[u'R', u'B', u'A', u'Q', u'D', u'V', u'P', u'I', u'S', u'J']
[u'U', u'L', u'Z', u'X', u'M', u'O', u'H', u'W', u'E', u'T']
[u'T', u'U', u'L', u'Z', u

In [14]:
for n in range(26):
    print dict_to_cycle(D36,n)

[u'A', u'J', u'I', u'G', u'O', u'E', u'D']
[u'C', u'Y', u'H', u'P', u'U', u'W', u'F']
[u'B', u'Q', u'K', u'S', u'N', u'L']
[u'E', u'D', u'A', u'J', u'I', u'G', u'O']
[u'D', u'A', u'J', u'I', u'G', u'O', u'E']
[u'G', u'O', u'E', u'D', u'A', u'J', u'I']
[u'F', u'C', u'Y', u'H', u'P', u'U', u'W']
[u'I', u'G', u'O', u'E', u'D', u'A', u'J']
[u'H', u'P', u'U', u'W', u'F', u'C', u'Y']
[u'K', u'S', u'N', u'L', u'B', u'Q']
[u'J', u'I', u'G', u'O', u'E', u'D', u'A']
[u'M', u'V', u'R', u'T', u'Z', u'X']
[u'L', u'B', u'Q', u'K', u'S', u'N']
[u'O', u'E', u'D', u'A', u'J', u'I', u'G']
[u'N', u'L', u'B', u'Q', u'K', u'S']
[u'Q', u'K', u'S', u'N', u'L', u'B']
[u'P', u'U', u'W', u'F', u'C', u'Y', u'H']
[u'S', u'N', u'L', u'B', u'Q', u'K']
[u'R', u'T', u'Z', u'X', u'M', u'V']
[u'U', u'W', u'F', u'C', u'Y', u'H', u'P']
[u'T', u'Z', u'X', u'M', u'V', u'R']
[u'W', u'F', u'C', u'Y', u'H', u'P', u'U']
[u'V', u'R', u'T', u'Z', u'X', u'M']
[u'Y', u'H', u'P', u'U', u'W', u'F', u'C']
[u'X', u'M', u'V', u'R', u'T

In [15]:
# So let's do it. First, generate the AD, BE, and CF permutation mappings. 
# You can do this by hand if you wish or you can write code to do so.

# If you do write code, use this cell to do so. You can use Ctrl + Enter/Return to run the code you write.
Dict={}
Dict['Z']='A'
Dict
Dict['Z']

'A'

In [16]:
# Now, find the cycles from the alphabet mappings. 
# Recall that these are of the form AD = (bfhjqkldeg)(capmvixuzr)(nop)(sty). 
# If you choose to write code, put it in this cell. 

Whew -- you've made it through a lot of hard work, and you're nearly there. With the cycles and their lengths in hand, you approach the trusty chain length catalog you spent years compiling. This catalog is indexed by cycle length and maps the AD, BE, and CF permutation cycle lengths to the rotor keys and rotor order that generated them. 

The index for this catalog (at least the catalog created for this exercise) is a little unique. It is a string of the chain lengths in numerical order prefaced by their respective dictionary name in alphabetical order. 

For example, suppose your AD mapping had cycles of length 2, 12, and 12; your BE mapping had cycles of  6, 8, and 10; and your CF mapping had cycles of length 4,6,8, and 8. The dictionary index you would search on would be 'AD:21212 BE:6810 CF:4688'. Not the prettiest, but it gets the job done. 

In [17]:
# First, let's load the chain dictionary and take a look at it. 
chains_dict = pickle.load(open('chains.pickle', 'rb'))

In [18]:
# To reference an item in the chains_dict, you'll want to use the following syntax: chains_dict[index]. Let's try it. 

#####EXAMPLE!!!!!!!#####################
chains_dict['AD:6677 BE:11221010 CF:221111']

[((u'A', u'A', u'B'), (u'II', u'III', u'I')),
 ((u'D', u'X', u'M'), (u'I', u'II', u'III')),
 ((u'E', u'I', u'W'), (u'II', u'I', u'III')),
 ((u'F', u'Q', u'X'), (u'I', u'III', u'II')),
 ((u'P', u'D', u'L'), (u'II', u'I', u'III')),
 ((u'P', u'V', u'H'), (u'II', u'III', u'I')),
 ((u'R', u'Y', u'E'), (u'II', u'I', u'III')),
 ((u'T', u'X', u'C'), (u'II', u'I', u'III')),
 ((u'U', u'M', u'W'), (u'I', u'II', u'III')),
 ((u'Z', u'S', u'A'), (u'I', u'III', u'II'))]

In [19]:
# Now, generate the index associated with the AD, BE, and CF cycles you found. 
# Once you've done that, look the cycles up in chains_dict using the indexing syntax described above.
# Note that the first item you get back represents the rotor key and the second item represents the rotor order
# from left to right. There will probably be more than one key/rotor order combination associated with the index.
chains_dict['AD:222299 BE:11221010 CF:6677']

[((u'A', u'A', u'C'), (u'II', u'III', u'I')),
 ((u'Y', u'A', u'Q'), (u'II', u'I', u'III'))]

### Good work -- you're almost there!
You've now pared down your search space from 105456 potential day key settings to 2. That's a massive reduction. Well done. 
Now you just need to figure out what the correct key/rotor setting is between these two and additionally figure out the plugboard settings. 

To assist you, there exist four files: encrypt1.txt, encrypt2.txt, encrypt3.txt, and encrypt4.txt. These are messages encrypted with the day key's rotor order and plugboard settings but the first message key's (the first one in the message_key_encrypts list, 'ZEUATW') rotor position, as would have been the case for a real German Enigma encryption. 

You have all the pieces available that you need. Good luck.

In [20]:
# It's probably a good idea to figure out what that twice encrypted message key really is. 

In [21]:
# Now, you can test if you found the right message key and also recover the plugboard settings. 

# This code will take in a rotor position, rotor order, and swap setting 
# (you can specify None for the swaps if you don't know them).
# This Enigma setting will be used to decrypt the three previously mentioned encrypted files. 
# It is your job to look through these potential decryptions and try to find the real day key, 
# keeping in mind that you still will need to recover the plugboard settings. 

def test_putative_key(rotor_position, rotor_order, plugs):
    '''
    Given a list of potential day key/rotor settings from the chain dictionary, try to decrypt files. 
    
    Rotor position = some three letter string i.e 'ABC'
    Rotor order = list of rotors from left to right i.e. ['I', 'III', 'II]
    Plugs = List of letters swapped (can be None or empty) i.e [] or ['AD', 'UY', 'IO']
    '''
    with open('encrypt1.txt', 'r') as f: 
        e1 = f.read()
    with open('encrypt2.txt', 'r') as f:
        e2 = f.read()
    with open('encrypt3.txt', 'r') as f:
        e3 = f.read()
    e = m.Enigma()
    e.set_rotor_order(rotor_order)
    e.set_plugs(plugs)
    e.set_rotor_position(rotor_position)
    d1 = e.decipher(e1)
    e.set_rotor_position(rotor_position)
    d2 = e.decipher(e2)
    e.set_rotor_position(rotor_position)
    d3 = e.decipher(e3)
    print('Rotor position: ' + rotor_position + ', rotor order: ' + str(rotor_order) + ', plugboard: ' + str(plugs) + ': ')
    print(d1)
    print(d2)
    print(d3)
    print('\n')

In [22]:
# Try out your potential Enigma settings here.
#test_putative_key('YAQ', ['II','I','III'],[])
e.set_rotor_position('YAQ')
e.set_rotor_order(['II','I','III'])

In [23]:
e.decipher('ZEUATW')

'TDSTDS'

In [27]:
# Try out your potential Enigma settings here.
test_putative_key('TDS', ['II','I','III'],['YH','FN','JS']) #['YH','FN','JS']

Rotor position: TDS, rotor order: ['II', 'I', 'III'], plugboard: ['YH', 'FN', 'JS']: 
ACALMANDMODESTLIFEBRINGSMOREHAPPINESSTHANTHEPURSUITOFSUCCESSCOMBINEDWITHCONSTANTRESTLESSNESSALBERTEINSTEIN
THEWAYTOMAKEPEOPLETRUSTWORTHYISTOTRUSTTHEMERNESTHEMMINGWAY
THEFUTUREBELONGSTOTHOSEWHOBELIEVEINTHEBEAUTYOFTHEIRDREAMSELEANORROOSEVELT




### CONGRATS! 
If you reach this cell, you've done it. Your success in cracking today's Enigma day key is no small feat. Well done. 